[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/GEE-Courses/blob/master/docs/gee_intro/Image/conditional_operations.ipynb)

**Relational, conditional and Boolean operations**

Reference: <https://developers.google.com/earth-engine/guides/image_relational>

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map(basemap="HYBRID")

To perform per-pixel comparisons between images, use relational operators. To extract urbanized areas in an image, this example uses relational operators to threshold spectral indices, combining the thresholds with `And()`:

In [ ]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

# Mask and display the binary layer.
Map.setCenter(-122.3578, 37.7726, 12)
Map.addLayer(bare.selfMask(), {}, 'bare')

Map

As illustrated by this example, the output of relational and boolean operators is either True (1) or False (0). To mask the 0's, you can mask the resultant binary image with itself. 

The binary images that are returned by relational and boolean operators can be used with mathematical operators. This example creates zones of urbanization in a nighttime lights image using relational operators and `image.add()`:

In [ ]:
Map = geemap.Map(basemap="HYBRID")

# Load a 2012 nightlights image.
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')
Map.addLayer(lights, {}, 'Nighttime lights')

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
Map.setCenter(2.373, 48.8683, 8)
Map.addLayer(zones, {'min': 0, 'max': 3, 'palette': palette}, 'development zones')

Map

Note that the code in the previous example is equivalent to using a [ternary operator](http://en.wikipedia.org/wiki/%3F:) implemented by `expression()`:

In [ ]:
Map = geemap.Map(basemap="HYBRID")

# Create zones using an expression, display.
zonesExp = nl2012.expression(
    "(b('stable_lights') > 62) ? 3"
    + ": (b('stable_lights') > 55) ? 2"
    + ": (b('stable_lights') > 30) ? 1"
    + ": 0"
)
Map.addLayer(
    zonesExp, {'min': 0, 'max': 3, 'palette': palette}, 'development zones (ternary)'
)
Map.setCenter(2.373, 48.8683, 8)

Map

Observe that in the previous expression example, the band of interest is referenced using the`b()` function, rather than a dictionary of variable names. (Learn more about image expressions on [this page](https://developers.google.com/earth-engine/image_math#expressions). Using either mathematical operators or an expression, the output is the same and should look something like Figure 2.

Another way to implement conditional operations on images is with the `image.where()` operator. Consider the need to replace masked pixels with some other data. In the following example, cloudy pixels are replaced by pixels from a cloud-free image using `where()`:

In [ ]:
Map = geemap.Map(basemap="HYBRID")

# Load a cloudy Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130603')
Map.addLayer(
    image, {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5}, 'original image'
)

# Load another image to replace the cloudy pixels.
replacement = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130416')

# Compute a cloud score band.
cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')
Map.addLayer(cloud, {}, 'Cloud score')

# Set cloudy pixels to the other image.
replaced = image.where(cloud.gt(10), replacement)

# Display the result.
Map.centerObject(image, 9)
Map.addLayer(
    replaced, {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5}, 'clouds replaced'
)
Map

In this example, observe the use of the `simpleCloudScore()` algorithm. This algorithm ranks pixels by cloudiness on a scale of 0-100, with 100 most cloudy. Learn more about `simpleCloudScore()` on the [Landsat Algorithms page](https://developers.google.com/earth-engine/landsat#simple-cloud-score).